In [48]:
from scipy.sparse import coo_matrix
import numpy as np

In [51]:
def get_indexs(i: int, j: int, n: int) -> int:
    """
    This function returns the indices of the vector $u$
    """
    nodes_per_row = 2**n + 1

    indices = i * nodes_per_row + j

    return indices

In [52]:
def vec_b(n: int) -> np.floating:
    """
    This function returns the value of the right-hand side vector $b$
    """
    h = np.float_power(2, -n)
    nodes_per_row = 2**n + 1

    def inner_f(x, y):
        return (
            12 * np.power(x, 2) * np.power(y, 5)
            + 17 * (x**2 + y**2) * np.sin(x * y)
            + 20 * np.power(x, 4) * np.power(y, 3)
        ) * h**2

    b_ls = np.zeros(nodes_per_row**2)

    for i in range(nodes_per_row):
        for j in range(nodes_per_row):
            idx = get_indexs(i, j, n)
            if i == 0 or j == 0:
                b_ls[idx] = 0
            elif i == nodes_per_row - 1:
                y = j * h
                b_ls[idx] = np.power(y, 5) - 17 * np.sin(y)
            elif j == nodes_per_row - 1:
                x = i * h
                b_ls[idx] = np.power(x, 4) - 17 * np.sin(x)
            else:
                x = i * h
                y = j * h
                b_ls[idx] = inner_f(x, y)

    b_ls = np.reshape(b_ls, (nodes_per_row**2, 1))

    assert b_ls.shape == (nodes_per_row**2, 1)
    return b_ls

In [53]:
def mat_A(n: int) -> np.ndarray:
    """
    This function returns the value of the matrix $A$
    """
    nodes_per_row = 2**n + 1
    row = np.array([])
    col = np.array([])
    A_ls = np.array([])

    for i in range(nodes_per_row):
        for j in range(nodes_per_row):
            idx = get_indexs(i, j, n)
            if i == 0 or j == 0 or i == nodes_per_row - 1 or j == nodes_per_row - 1:
                row = np.append(row, idx)
                col = np.append(col, idx)
                A_ls = np.append(A_ls, 1)
            else:
                row = np.append(row, idx)
                col = np.append(col, idx)
                A_ls = np.append(A_ls, -4)
                for idx_ in [
                    get_indexs(i - 1, j, n),
                    get_indexs(i + 1, j, n),
                    get_indexs(i, j - 1, n),
                    get_indexs(i, j + 1, n),
                ]:
                    row = np.append(row, idx)
                    col = np.append(col, idx_)
                    A_ls = np.append(A_ls, 1)

    A = coo_matrix(
        (A_ls, (row, col)), shape=(nodes_per_row**2, nodes_per_row**2)
    )
    assert A.shape == (nodes_per_row**2, nodes_per_row**2)
    return A

In [54]:
mat_A(1).toarray()

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1., -4.,  1.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [55]:
vec_b(1)

array([[  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.58823341],
       [ -8.08773416],
       [  0.        ],
       [ -8.11898416],
       [-13.30500674]])